In [1]:
import cooler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import h5py
import pickle
import seaborn as sns
import tqdm

from scipy import stats

from statsmodels.stats.multitest import multipletests

import sys
import itertools
import time
import datetime
import re

sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
from __init__ import *
from __init__jupyterlab import * 
import snmcseq_utils

from enhancer_gene_hic_validation_utils import *

In [2]:
# default_colorbar_kws = DEFAULT_COLORBAR_KWS
today = datetime.date.today()
print(today)

2021-01-08


In [3]:
chromosomes = ['chr'+str(i+1) for i in range(22)] + ['chrX']
celltypes = ['L2_3', 'L4', 'L5', 'L6', 
             'Pvalb', 'Sst', 'Vip', 'Ndnf']
resolution = 10000

# HiC-analysis - use only cell type-specific pairs

In [4]:
# gene expression profiles
f = '/cndd2/fangming/sphere_old/enhancers/scripts/data_organized/gene_rna_clsts.tsv'
gene_rna_clsts = pd.read_csv(f, sep='\t', index_col=0) #header=True, index=True)

# cluster annotation
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_ethan38_200520/ref/annotations_order.tsv'
clst_annot = pd.read_csv(f, sep='\t').set_index('cluster')['annotation']

# merge cluster name
annot_rename_lookup = {
    'L2/3': "L2_3",
    'L4/5': "L4",
    'L6b': "L6",
    'Sncg': "Ndnf",
    'Lamp5': "Ndnf",
}

def rename_annot(x, annot_rename_lookup):
    """
    """
    if x in annot_rename_lookup.keys():
        return annot_rename_lookup[x]
    else:
        return x
    
clst_annot_broad = pd.Series([annot.split(' ')[0] for annot in clst_annot.values], 
                             index=clst_annot.index)
clst_annot_broad = clst_annot_broad.apply(lambda x: rename_annot(x, annot_rename_lookup))

print(clst_annot_broad.shape, 
      len(np.unique(clst_annot_broad.values)),
      np.unique(clst_annot_broad.values),
     )

(38,) 8 ['L2_3' 'L4' 'L5' 'L6' 'Ndnf' 'Pvalb' 'Sst' 'Vip']


In [5]:
# merge RNA profiles
_tmp = gene_rna_clsts.T
_tmp['annot'] = clst_annot_broad.loc[_tmp.index]
gene_rna_clsts_broad = _tmp.groupby('annot').median().T
gene_rna_clsts_broad_rank = gene_rna_clsts_broad.rank(pct=True, axis=1)
print(gene_rna_clsts_broad_rank.shape)
gene_rna_clsts_broad_rank.head()

(12095, 8)


annot,L2_3,L4,L5,L6,Ndnf,Pvalb,Sst,Vip
ENSMUSG00000000001,0.500,0.750,0.250,0.625,0.875,0.125,0.375,1.000
ENSMUSG00000000056,0.125,0.750,0.875,0.500,0.625,1.000,0.250,0.375
ENSMUSG00000000058,0.625,1.000,0.250,0.375,0.750,0.125,0.500,0.875
ENSMUSG00000000078,0.250,1.000,0.625,0.750,0.125,0.500,0.875,0.375
ENSMUSG00000000085,0.250,0.125,0.625,0.375,0.875,0.500,0.750,1.000


In [6]:
active_th = 0.50

# min_matches = np.arange(0.1, 1.00+0.05, 0.05)
min_matches = [0.8]
print(min_matches)

[0.8]


In [7]:
# enhancers lifted

for min_match in tqdm.tqdm(min_matches):
    print("**********{}".format(min_match))

    output_res_format = ('/cndd2/fangming/sphere_old/enhancers/scripts/results_new/'
                  'hic_contact_enrichment_metacell_liftover{:.2f}_celltypespec_{{}}_{{}}_{{}}_{{}}.pkl' #.format(int(100*active_th), celltype, resolution, today)
                  .format(min_match)
                 )

    f = '/cndd2/fangming/projects/scf_enhancers/enhancer_metacells_r100_n935/liftover/enhs_list_reformat_minmatch_{:.2f}_lifted.bed'.format(min_match)
    enh_list = pd.read_csv(f, sep='\t', header=None, names=['chr', 'start', 'end', 'enh_id']).astype({'chr': str, 
                                                                                                      'start': int,
                                                                                                      'end': int,
                                                                                                      'enh_id': int,
                                                                                                     })
    enh_list['center'] = (enh_list['start'] + enh_list['end'])/2

    # tss lifted
    f = '/cndd2/fangming/projects/scf_enhancers/enhancer_metacells_r100_n935/liftover/gencode.vM16.annotation_genes_tss_lifted.bed'
    tss_list = pd.read_csv(f, sep='\t', header=None, names=['chr', 'start', 'end', 'gid', 'gene_name'])
    tss_list['gid'] = tss_list['gid'].apply(lambda x: x.split('.')[0])

    # enh - tss linked list
    # orig significant pairs (enh_index and gid)
    input_format = '/cndd2/fangming/projects/scf_enhancers/enhancer_metacells_r100_n935/results_new/fig2_metacell_201227_r25.1_n256_'
    f = input_format + 'linked_pairs_mc.tsv'
    linked_pairs_mc = pd.read_csv(f, sep="\t")

    f = input_format + 'linked_pairs_atac.tsv'
    linked_pairs_atac = pd.read_csv(f, sep="\t")

    f = input_format + 'correlated_pairs_mc.tsv'
    correlated_pairs_mc = pd.read_csv(f, sep="\t")

    f = input_format + 'correlated_pairs_atac.tsv'
    correlated_pairs_atac = pd.read_csv(f, sep="\t")
    
    # subtract linked from correlated
    # genes can be divided into 4 non-overlapping quardrone: linked vs correlated; mc vs atac
    # below groups the 4 quardrone into 3 different groups
    correlated_pairs_mc = (pd.merge(linked_pairs_mc, correlated_pairs_mc, how='right', indicator=True)
                             .query('_merge=="right_only"')
                             .drop('_merge', axis=1)
                          )
    correlated_pairs_atac = (pd.merge(linked_pairs_atac, correlated_pairs_atac, how='right', indicator=True)
                             .query('_merge=="right_only"')
                             .drop('_merge', axis=1)
                          )
#     correlated_pairs_both = (pd.merge(linked_pairs_both, correlated_pairs_both, how='right', indicator=True)
#                              .query('_merge=="right_only"')
#                              .drop('_merge', axis=1)
#                           )
    linked_pairs_both = pd.merge(linked_pairs_mc, linked_pairs_atac, on=['gene', 'enh', 'dist'])
    correlated_pairs_both = pd.merge(correlated_pairs_mc, correlated_pairs_atac, on=['gene', 'enh', 'dist'])

    # distances
    distance_cutoff = 1e5
    distance_idx = np.arange(1, int(distance_cutoff/resolution)+1)
    distances = distance_idx * resolution

    for celltype in celltypes:
        print(celltype)

        contacts = [[] for i in np.arange(len(distances))]
        contacts_sig_pairs = collections.OrderedDict({
                'linked_mc': [[] for i in np.arange(len(distances))],
                'linked_atac': [[] for i in np.arange(len(distances))],
                'linked_both': [[] for i in np.arange(len(distances))],
                'correlated_mc': [[] for i in np.arange(len(distances))],
                'correlated_atac': [[] for i in np.arange(len(distances))],
                'correlated_both': [[] for i in np.arange(len(distances))],
            })

        for chromosome in chromosomes:
            print(celltype, resolution, chromosome)

            # read in the matrix 
            resource_string = (
                 '/cndd2/fangming/sphere_old/enhancers/data_m3c/{}_all_brain.txt_1kb_contacts.mcool'
                 '::/resolutions/{}'.format(celltype, resolution)
                 )
            c = cooler.Cooler(resource_string)
            mat_all = c.matrix(balance=True).fetch(chromosome)

            # all enhancers and tss on that chromosomes
            enhs = enh_list[enh_list['chr']==chromosome]
            tsss = tss_list[tss_list['chr']==chromosome]

            # only include the gene if it is actively expressed (>50% across cell types)
            active_genes = gene_rna_clsts_broad_rank[gene_rna_clsts_broad_rank[celltype]>active_th].index.values
            tsss = tsss[tsss['gid'].isin(active_genes)]
            
            

            # get paired bins
            paired_bin_tables = collections.OrderedDict({
                'linked_mc': enh_gene_id_to_binidx(linked_pairs_mc, tsss, enhs, resolution),
                'linked_atac': enh_gene_id_to_binidx(linked_pairs_atac, tsss, enhs, resolution),
                'linked_both': enh_gene_id_to_binidx(linked_pairs_both, tsss, enhs, resolution),
                'correlated_mc': enh_gene_id_to_binidx(correlated_pairs_mc, tsss, enhs, resolution),
                'correlated_atac': enh_gene_id_to_binidx(correlated_pairs_atac, tsss, enhs, resolution),
                'correlated_both': enh_gene_id_to_binidx(correlated_pairs_both, tsss, enhs, resolution),
            })
            track_names = list(paired_bin_tables.keys())

            # contact stats: distance bin by corr values in that bin
            (_contacts, _contacts_sig_pairs
            ) = get_contacts(mat_all, paired_bin_tables, resolution, distance_idx)

            # combine all chromosomes 
            for idx, dist in enumerate(distances):
                contacts[idx] += list(_contacts[idx])
                for key in _contacts_sig_pairs.keys():
                    contacts_sig_pairs[key][idx] += list(_contacts_sig_pairs[key][idx])
    #         break

        # stats
        contacts_mean = np.array([np.nanmean(val) for val in contacts])
        contacts_std = np.array([np.nanstd(val) for val in contacts])
        contacts_n = np.array([len(val) for val in contacts])

        contacts_sig_pairs_mean = collections.OrderedDict({
            key: np.zeros(len(distances)) 
            for key in contacts_sig_pairs.keys()
        })
        contacts_sig_pairs_std = contacts_sig_pairs_mean.copy()
        contacts_sig_pairs_n = contacts_sig_pairs_mean.copy()

        for key in contacts_sig_pairs.keys():
            contacts_sig_pairs_mean[key] = np.array([np.nanmean(val) 
                                                     for val in contacts_sig_pairs[key]])

            contacts_sig_pairs_std[key] = np.array([np.nanstd(val) 
                                                    for val in contacts_sig_pairs[key]])

            contacts_sig_pairs_n[key] = np.array([len(val) 
                                                  for val in contacts_sig_pairs[key]])


        # run and save results
        output_res = output_res_format.format(int(100*active_th), celltype, resolution, today)
        print(output_res)

        # save results
        # distances, contacts_mean, contacts_std ## these are numpy arrays 
        # contacts_sig_pairs_mean, contacts_sig_pairs_std ## these
        with open(output_res, 'wb') as fwh:
            pickle.dump((distances, 
                         contacts_mean, contacts_std, contacts_n,
                         contacts_sig_pairs_mean, 
                         contacts_sig_pairs_std,
                         contacts_sig_pairs_n,
                        ), fwh)

  0%|          | 0/1 [00:00<?, ?it/s]

**********0.8
L2_3
L2_3 10000 chr1



100%|██████████| 10/10 [00:00<00:00, 271.82it/s]


L2_3 10000 chr2



100%|██████████| 10/10 [00:00<00:00, 296.24it/s]


...

Ndnf 10000 chr22



100%|██████████| 10/10 [00:00<00:00, 429.44it/s]


Ndnf 10000 chrX



100%|██████████| 1/1 [06:10<00:00, 370.05s/it]

/cndd2/fangming/sphere_old/enhancers/scripts/results_new/hic_contact_enrichment_metacell_liftover0.80_celltypespec_50_Ndnf_10000_2021-01-08.pkl
